In [1]:
import qiskit
print(f"Qiskit version: {qiskit.__version__}")

# You should see something like: Qiskit version: 2.x.x

Qiskit version: 2.3.0


In [3]:
from qiskit import QuantumCircuit

# Create a circuit with 2 qubits and 2 classical bits
qc = QuantumCircuit(2, 2)

# Step 1: Apply Hadamard to qubit 0 (create superposition)
qc.h(0)

# Step 2: Apply CNOT (entangle qubit 0 and qubit 1)
qc.cx(0, 1)

# Step 3: Measure both qubits
qc.measure([0, 1], [0, 1])

# Draw the circuit
print(qc.draw())

     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 


In [3]:
# Circuit properties
print(f"Number of qubits: {qc.num_qubits}")       # 2
print(f"Number of classical bits: {qc.num_clbits}") # 2
print(f"Circuit depth: {qc.depth()}")               # 3
print(f"Gate counts: {qc.count_ops()}")             # {'h': 1, 'cx': 1, 'measure': 2}

Number of qubits: 2
Number of classical bits: 2
Circuit depth: 3
Gate counts: OrderedDict({'measure': 2, 'h': 1, 'cx': 1})


In [5]:
from qiskit import QuantumCircuit
import numpy as np

qc = QuantumCircuit(3, 3)

# Single-qubit gates
qc.x(0)           # Pauli-X (NOT) on qubit 0
qc.y(1)           # Pauli-Y on qubit 1
qc.z(2)           # Pauli-Z on qubit 2
qc.h(0)           # Hadamard on qubit 0
qc.s(1)           # S gate on qubit 1
qc.t(2)           # T gate on qubit 2

# Rotation gates
qc.rx(np.pi, 0)    # Rotate around X by pi
qc.ry(np.pi/2, 1)    # Rotate around Y by pi/2
qc.rz(0.78, 2)    # Rotate around Z

# Multi-qubit gates
qc.cx(0, 1)       # CNOT: control=0, target=1
qc.cz(1, 2)       # CZ: control=1, target=2
qc.ccx(0, 1, 2)   # Toffoli: controls=0,1, target=2
qc.swap(0, 1)     # SWAP qubits 0 and 1

# Barriers (visual separators)
qc.barrier()

# Measurement
qc.measure([0, 1, 2], [0, 1, 2])

print(qc.draw())

     ┌───┐┌───┐ ┌───────┐                   ░ ┌─┐      
q_0: ┤ X ├┤ H ├─┤ Rx(π) ├────■───────■───X──░─┤M├──────
     ├───┤├───┤┌┴───────┴┐ ┌─┴─┐     │   │  ░ └╥┘┌─┐   
q_1: ┤ Y ├┤ S ├┤ Ry(π/2) ├─┤ X ├─■───■───X──░──╫─┤M├───
     ├───┤├───┤├─────────┴┐└───┘ │ ┌─┴─┐    ░  ║ └╥┘┌─┐
q_2: ┤ Z ├┤ T ├┤ Rz(0.78) ├──────■─┤ X ├────░──╫──╫─┤M├
     └───┘└───┘└──────────┘        └───┘    ░  ║  ║ └╥┘
c: 3/══════════════════════════════════════════╩══╩══╩═
                                               0  1  2 


In [6]:
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

# Build circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

# Run on simulator
sampler = StatevectorSampler()
job = sampler.run([qc], shots=1024)
result = job.result()

# Get measurement counts
counts = result[0].data.c.get_counts()
print(counts)
# Output: {'00': 512, '11': 512}  (approximately)

{'11': 507, '00': 517}


In [ ]:
from qiskit.visualization import plot_histogram

# Plot a histogram of results
fig = plot_histogram(counts)
fig.savefig('results.png')

In [14]:

from qiskit_ibm_runtime import QiskitRuntimeService

# Save your account (only need to do this once)
QiskitRuntimeService.save_account(
    channel="ibm_quantum_platform",
    token="fBSlLGMrl1uwzWdFIHJa9dzRVNGezhH0UCwoGNdvZPDx",
    overwrite=True
)

# Verify it works
service = QiskitRuntimeService()
print("Available backends:")
for backend in service.backends():
    print(f"  {backend.name} - {backend.num_qubits} qubits")



qiskit_runtime_service.__init__:WARNING:2026-02-10 19:49:08,953: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2026-02-10 19:49:08,954: Loading instance: open-instance, plan: open


Available backends:
  ibm_fez - 156 qubits
  ibm_torino - 133 qubits
  ibm_marrakesh - 156 qubits


In [15]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Build your ideal circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

# Transpile for the specific backend
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
transpiled_qc = pm.run(qc)

print(f"Original depth: {qc.depth()}")
print(f"Transpiled depth: {transpiled_qc.depth()}")
print(f"Original gates: {qc.count_ops()}")
print(f"Transpiled gates: {transpiled_qc.count_ops()}")

Original depth: 3
Transpiled depth: 7
Original gates: OrderedDict({'measure': 2, 'h': 1, 'cx': 1})
Transpiled gates: OrderedDict({'rz': 5, 'sx': 3, 'measure': 2, 'cz': 1})


In [16]:
from qiskit_ibm_runtime import SamplerV2

sampler = SamplerV2(backend)
job = sampler.run([transpiled_qc], shots=4096)

# Wait for results (real hardware takes seconds to minutes)
result = job.result()
counts = result[0].data.c.get_counts()
print(counts)
# Output: {'00': 1985, '11': 1943, '01': 87, '10': 81}
#          Notice small counts for 01/10 -- this is HARDWARE NOISE!

{'00': 2106, '11': 1950, '01': 16, '10': 24}


In [18]:
"""
Complete Qiskit Example: Creating and Analyzing a GHZ State
A GHZ state is a 3-qubit entangled state: (|000> + |111>)/sqrt(2)
"""

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
from qiskit.quantum_info import Statevector

# --- BUILD THE CIRCUIT ---
qc = QuantumCircuit(3, 3)

# Create GHZ state
qc.h(0)          # Superposition on qubit 0
qc.cx(0, 1)      # Entangle qubit 0 -> 1
qc.cx(0, 2)      # Entangle qubit 0 -> 2
qc.barrier()
qc.measure([0, 1, 2], [0, 1, 2])

print("=== GHZ State Circuit ===")
print(qc.draw())
print(f"\nCircuit depth: {qc.depth()}")
print(f"Gate counts: {qc.count_ops()}")

# --- ANALYZE THE STATE (before measurement) ---
qc_no_measure = QuantumCircuit(3)
qc_no_measure.h(0)
qc_no_measure.cx(0, 1)
qc_no_measure.cx(0, 2)

statevector = Statevector.from_instruction(qc_no_measure)
print(f"\n=== Statevector ===")
print(statevector)
# Shows: (|000> + |111>)/sqrt(2)

# --- SIMULATE ---
sampler = StatevectorSampler()
job = sampler.run([qc], shots=40096)
result = job.result()
counts = result[0].data.c.get_counts()

print(f"\n=== Measurement Results (4096 shots) ===")
total = sum(counts.values())
for state in sorted(counts.keys()):
    count = counts[state]
    prob = count / total
    bar = '#' * int(prob * 50)
    print(f"|{state}>: {count:4d} ({prob:6.2%}) {bar}")

# Expected output:
# |000>: ~2048 (50%)  #########################
# |111>: ~2048 (50%)  #########################
# All other states: 0 (0%)

=== GHZ State Circuit ===
     ┌───┐           ░ ┌─┐      
q_0: ┤ H ├──■────■───░─┤M├──────
     └───┘┌─┴─┐  │   ░ └╥┘┌─┐   
q_1: ─────┤ X ├──┼───░──╫─┤M├───
          └───┘┌─┴─┐ ░  ║ └╥┘┌─┐
q_2: ──────────┤ X ├─░──╫──╫─┤M├
               └───┘ ░  ║  ║ └╥┘
c: 3/═══════════════════╩══╩══╩═
                        0  1  2 

Circuit depth: 4
Gate counts: OrderedDict({'measure': 3, 'cx': 2, 'h': 1, 'barrier': 1})

=== Statevector ===
Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.70710678+0.j],
            dims=(2, 2, 2))

=== Measurement Results (4096 shots) ===
|000>: 20081 (50.08%) #########################
|111>: 20015 (49.92%) ########################


In [20]:

"""
Simulating the Hydrogen molecule (H2) energy curve
This shows how bond distance affects molecular energy
"""
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import StatevectorEstimator
import numpy as np
from scipy.optimize import minimize

def run_vqe(hamiltonian_coefficients):
    """Run VQE for a given Hamiltonian"""
    hamiltonian = SparsePauliOp.from_list(hamiltonian_coefficients)
    
    theta = Parameter('theta')
    ansatz = QuantumCircuit(2)
    ansatz.ry(theta, 0)
    ansatz.cx(0, 1)
    
    estimator = StatevectorEstimator()
    
    def cost(params):
        bound = ansatz.assign_parameters({theta: params[0]})
        job = estimator.run([(bound, hamiltonian)])
        return float(job.result()[0].data.evs)
    
    result = minimize(cost, x0=[0.0], method='COBYLA')
    return result.fun

# H2 Hamiltonian at equilibrium bond distance (0.735 Angstroms)
h2_hamiltonian = [
    ("II", -1.052),
    ("IZ",  0.398),
    ("ZI", -0.398),
    ("ZZ", -0.011),
    ("XX",  0.181)
]

energy = run_vqe(h2_hamiltonian)
print(f"H2 ground state energy: {energy:.6f} Hartree")
print(f"Expected (exact):       -1.137284 Hartree")

H2 ground state energy: -1.244000 Hartree
Expected (exact):       -1.137284 Hartree
